## Calculate Phi vertical mean for a particular layer and LME

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import pandas as pd
import utils
import warnings
warnings.filterwarnings('ignore')
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.pyplot as plt

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
a1,a2,b1,b2 = [240,380,125,320]

In [3]:
datadir = '/glade/campaign/cesm/collections/CESM1-DPLE/DPLE-FOSI/rerun_with_extension/timeseries'
filename = f'{datadir}/g.DPLE.GECOIAF.T62_g16.009.chey.pop.h.TEMP.024901-031812.nc'
dset = xr.open_dataset(filename)
dz = dset['dz'].isel(z_t=range(0, 35))/1e2 # m
darea = dset['TAREA'].isel(nlat=range(a1, a2), nlon=range(b1, b2))/1e10 #km*km
KMT = dset['KMT'].isel(nlat=range(a1, a2), nlon=range(b1, b2))
dset.close()

In [4]:
dvol = dz*darea
for xx in range(a2-a1):
    for yy in range(b2-b1):
        kk = np.int(KMT[xx, yy])
        if kk < 60:
            dvol[kk:, xx, yy] = float("nan")

In [24]:
#varr = 'O2'
#varr = 'AOU'
#varr = 'TEMP'
varr = 'SALT'

In [35]:
filename = f'{path}/DPLE_driftcorrected_Phi_components_{varr}_ens_mean_monthly.nc'
ds = xr.open_dataset(filename)
dsf = xr.Dataset()
for ii in [0,6,12]:
    Phi_i = ds[f'Phi_{varr}_{ii}']*dvol
    # select a vertical layer
    dsf[f'Phi_{varr}_{ii}_0-200m'] = Phi_i.isel(z_t=range(0, 20, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(0, 20, 1)).sum(dim='z_t'))
    dsf[f'Phi_{varr}_{ii}_200-600m'] = Phi_i.isel(z_t=range(20, 35, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(20, 35, 1)).sum(dim='z_t'))

In [36]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:              (nlat: 140, nlon: 195, Y: 64, L: 122)
Coordinates:
    ULONG                (nlat, nlon) float64 101.7 102.9 104.0 ... 319.5 320.0
    ULAT                 (nlat, nlon) float64 15.13 15.13 15.14 ... 70.63 70.63
  * Y                    (Y) int64 1954 1955 1956 1957 ... 2014 2015 2016 2017
  * L                    (L) int64 1 2 3 4 5 6 7 ... 116 117 118 119 120 121 122
    TLAT                 (nlat, nlon) float64 14.96 14.96 14.97 ... 70.42 70.42
    TLONG                (nlat, nlon) float64 101.2 102.3 103.4 ... 319.2 319.7
Dimensions without coordinates: nlat, nlon
Data variables:
    Phi_SALT_0_0-200m    (L, nlat, nlon, Y) float64 nan nan nan ... nan nan nan
    Phi_SALT_0_200-600m  (L, nlat, nlon, Y) float64 nan nan nan ... nan nan nan
    Phi_SALT_1_0-200m    (L, nlat, nlon, Y) float64 nan nan nan ... nan nan nan
    Phi_SALT_1_200-600m  (L, nlat, nlon, Y) float64 nan nan nan ... nan nan nan
    Phi_SALT_2_0-200m    (L, nlat, nlon, Y) float64 nan nan nan ... nan nan nan
    Phi_SALT_2_200-600m  (L, nlat, nlon, Y) float64 nan nan nan ... nan nan nan

In [37]:
%time
#dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_vertical_mean_ens_mean_monthly.nc', mode='w')
dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_components_{varr}_vertical_mean_ens_mean_monthly.nc', mode='w')

CPU times: user 1e+03 ns, sys: 2 µs, total: 3 µs
Wall time: 5.72 µs
